#### Reverse-Engineering Binary WASM

In [30]:
def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i, end=' ')
    def writeln(s): print()
    def read(s): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

In [31]:
def hexdump(fn: str):
    with open(fn, 'rb') as hexfile: # open binary file for reading
        word, pos = hexfile.read(4), 0 # type(word) == bytes
        while len(word) > 0:
            print('{:#08x}'.format(pos) + ': ' +
                  ' '.join('{:02X}'.format(b) for b in word) + '    ' +
                  ''.join(chr(c) if 32 <= c < 127 else '.' for c in word))
            word, pos = hexfile.read(4), pos + 4

In [32]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString

Consider the following P0 program. Compile it first to a textual .wat file and then convert it to a binary file:

In [33]:
compileString("""
const N = 5
var a: [1 .. N] → integer
program seven
  var i : integer
    a[1] := 3; a[2] := 7; a[3] := 5; a[4] := 9; a[5] := 7
    i := 1
    while i ≤ N do
      if a[i] = 7 then write(i)
      i := i + 1
""", 'seven.wat')

In [34]:
!wat2wasm seven.wat

You may check that it works as intended by executing the binary file:

In [35]:
runpywasm('seven.wasm')

2 5 

In [36]:
!cat seven.wat

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(global $_memsize (mut i32) i32.const 20)
(func $program
(local $i i32)
(local $0 i32)
i32.const 0
i32.const 3
i32.store
i32.const 4
i32.const 7
i32.store
i32.const 8
i32.const 5
i32.store
i32.const 12
i32.const 9
i32.store
i32.const 16
i32.const 7
i32.store
i32.const 1
local.set $i
loop
local.get $i
i32.const 5
i32.le_s
if
local.get $i
i32.const 1
i32.sub
i32.const 4
i32.mul
i32.const 0
i32.add
i32.load
i32.const 7
i32.eq
if
local.get $i
call $write
end
local.get $i
i32.const 1
i32.add
local.set $i
br 1
end
end
)
(memory 1)
(start $program)
)

In [37]:
!wasm2wat seven.wasm

(module
  (type (;0;) (func (param i32)))
  (type (;1;) (func))
  (type (;2;) (func (result i32)))
  (import "P0lib" "write" (func (;0;) (type 0)))
  (import "P0lib" "writeln" (func (;1;) (type 1)))
  (import "P0lib" "read" (func (;2;) (type 2)))
  (func (;3;) (type 1)
    (local i32 i32)
    i32.const 0
    i32.const 3
    i32.store
    i32.const 4
    i32.const 7
    i32.store
    i32.const 8
    i32.const 5
    i32.store
    i32.const 12
    i32.const 9
    i32.store
    i32.const 16
    i32.const 7
    i32.store
    i32.const 1
    local.set 0
    loop  ;; label = @1
      local.get 0
      i32.const 5
      i32.le_s
      if  ;; label = @2
        local.get 0
        i32.const 1
        i32.sub
        i32.const 4
        i32.mul
        i32.const 0
        i32.add
        i32.load
        i32.const 7
        i32.eq
        if  ;; label = @3
          local.get 0
          call 0
        end
        local.get 0
        i32.const 1
        i32.add
        local.set 0
        br 1 (

Now "reverse engineer" `seven.wasm`:

In [38]:
hexdump('seven.wasm')

0x000000: 00 61 73 6D    .asm
0x000004: 01 00 00 00    ....
0x000008: 01 0C 03 60    ...`
0x00000c: 01 7F 00 60    ...`
0x000010: 00 00 60 00    ..`.
0x000014: 01 7F 02 2C    ...,
0x000018: 03 05 50 30    ..P0
0x00001c: 6C 69 62 05    lib.
0x000020: 77 72 69 74    writ
0x000024: 65 00 00 05    e...
0x000028: 50 30 6C 69    P0li
0x00002c: 62 07 77 72    b.wr
0x000030: 69 74 65 6C    itel
0x000034: 6E 00 01 05    n...
0x000038: 50 30 6C 69    P0li
0x00003c: 62 04 72 65    b.re
0x000040: 61 64 00 02    ad..
0x000044: 03 02 01 01    ....
0x000048: 05 03 01 00    ....
0x00004c: 01 06 06 01    ....
0x000050: 7F 01 41 14    ..A.
0x000054: 0B 08 01 03    ....
0x000058: 0A 59 01 57    .Y.W
0x00005c: 01 02 7F 41    ...A
0x000060: 00 41 03 36    .A.6
0x000064: 02 00 41 04    ..A.
0x000068: 41 07 36 02    A.6.
0x00006c: 00 41 08 41    .A.A
0x000070: 05 36 02 00    .6..
0x000074: 41 0C 41 09    A.A.
0x000078: 36 02 00 41    6..A
0x00007c: 10 41 07 36    .A.6
0x000080: 02 00 41 01    ..A.
0x000084: 

##### Part A

How long is the generated code in (decimal) bytes?

The length of the generated code is 179 bytes.

##### Part B

Annotate the hex dump of `seven.wasm` in the same way `max.wasm` above!

Here is an annotated version of the hex dump of `seven.wasm`:

<pre style="font-family:monospace;color:royalblue">
0x000000: 00 61 73 6D    .asm    magic
0x000004: 01 00 00 00    ....    version
0x000008: 01 0C 03 60    ...`    typesecid, size typesec, #types, functype (0)
0x00000c: 01 7F 00 60    ...`    typesecid, size typesec, #types, functype (0)
0x000010: 00 00 60 00    ..`.    #params, #results, functype (2), #params
0x000014: 01 7F 02 2C    ...,    #results, i32, importsecid, length import
0x000018: 03 05 50 30    ..P0    #imports, len(P0lib), 'P', '0'
0x00001c: 6C 69 62 05    lib.    'l', 'i', 'b', len('write')
0x000020: 77 72 69 74    writ    'w', 'r', 'i', 't'
0x000024: 65 00 00 05    e...    'e', typeidx, 0, len('P0lib')
0x000028: 50 30 6C 69    P0li    'P', '0', 'l', 'i'
0x00002c: 62 07 77 72    b.wr    'b', len('writeln'), 'w', 'r'
0x000030: 69 74 65 6C    itel    'i', 't', 'e', 'l'
0x000034: 6E 00 01 05    n...    'n', typeidx, 1, len('P0lib')
0x000038: 50 30 6C 69    P0li    'P', '0', 'l', 'i'
0x00003c: 62 04 72 65    b.re    'b', len('read'), 'r', 'e'
0x000040: 61 64 00 02    ad..    'a', 'd', typeidx, 2
0x000044: 03 02 01 01    ....    funcsec, length 2, #typeidx, type 1
0x000048: 05 03 01 00    ....    memsec, length 3, #mem, 0 (only min)
0x00004c: 01 06 06 01    ....    1 (min), globalsec, size globalsec, #globals
0x000050: 7F 01 41 14    ..A.    i32, var, i32.const, 20
0x000054: 0B 08 01 03    ....    end (expression), startsec, size startsec, 3 (function)
0x000058: 0A 59 01 57    .Y.W    codesection, sizecodesection, #functions, length code
0x00005c: 01 02 7F 41    ...A    1 (num locals), 2 (num vars), type (int32), i32.const
0x000060: 00 41 03 36    .A.6    0, i32.const, 3, i32.store
0x000064: 02 00 41 04    ..A.    2 (align), 0 (offset), i32.const, 4
0x000068: 41 07 36 02    A.6.    i32.const, 7, i32.store, 2 (align)
0x00006c: 00 41 08 41    .A.A    0 (offset), i32.const, 8, i32.const
0x000070: 05 36 02 00    .6..    5, i32.store, 2 (align), 0 (offset)
0x000074: 41 0C 41 09    A.A.    i32.const, 12, i32.const, 9
0x000078: 36 02 00 41    6..A    i32.store, 2 (align), 0 (offset), i32.const
0x00007c: 10 41 07 36    .A.6    16, i32.const, 7, i32.store
0x000080: 02 00 41 01    ..A.    2 (align), 0 (offset), i32.const, 1
0x000084: 21 00 03 40    !..@    local.set, 0 (i), loop, [] (empty type)
0x000088: 20 00 41 05     .A.    local.get, 0 (i), i32.const, 5
0x00008c: 4C 04 40 20    L.@     i32.le_s, if, [] (empty type), local.get
0x000090: 00 41 01 6B    .A.k    0 (i), i32.const, 1, i32.sub
0x000094: 41 04 6C 41    A.lA    i32.const, 4, i32.mul, i32.const
0x000098: 00 6A 28 02    .j(.    0, i32.add, i32.load, 2
0x00009c: 00 41 07 46    .A.F    0, i32.const, 7, i32.eq
0x0000a0: 04 40 20 00    .@ .    if, [] (empty type), local.get, 0 (i)
0x0000a4: 10 00 0B 20    ...     call, 0 (write), end, local.get
0x0000a8: 00 41 01 6A    .A.j    0 (i), i32.const, 1, i32.add
0x0000ac: 21 00 0C 01    !...    local.set, 0 (i), br, 1
0x0000b0: 0B 0B 0B    ...        end, end, end
</pre>